### Python-SQL Connection

In [81]:
import psycopg2
import numpy as np
import pandas as pd
import os

In [48]:
class Connection:
    def __init__(self,name_db):
        
        self.conn = psycopg2.connect(dbname = f'{name_db}', 
                            user = 'melton_zheng', 
                            password = '8FbxU0hT(CgcUM9O',
                            host = '34.72.27.131')

        self.cursor = self.conn.cursor()
        self.users = self.sql_to_df('users')
        self.roles = self.sql_to_df('roles')
        self.role_map = self.sql_to_df('provider_org_role_maps')
        self.chat_data = self.sql_to_df('chat_datas')

    def sql_to_df(self,name_of_table):
        self.cursor.execute(f'SELECT * FROM public.{name_of_table};')
        d = self.cursor.fetchall()
        self.cursor.execute(f'SELECT * FROM public.{name_of_table} LIMIT 0;')
        c = [desc[0] for desc in self.cursor.description]
        _df = pd.DataFrame(data=d, columns=c)
        return _df
    
    def find_user(self,_id):
        ids = self.users['user_id']
        i = ids.tolist().index(_id)
        if(i != -1):
            return self.users.loc[i]
        else:
            print('Could not find patient in user list')
            return
        
    def find_role(self,_id):
        q = self.role_map['provider_id'].to_numpy()
        w = np.where(q==_id)
        #print(w)
        e = self.role_map['roles'].to_numpy()[w]
        r = self.roles['role_id'].to_numpy()
        try:
            t = np.where(r==e[0][1:-1])
            # print(t)
            y = self.roles['role_name'].to_numpy()
            return(y[t])
        except:
            # print('Could not find the physician in role map')
            return None
        
    
    def make_patient(self,patient_id):
    
        u = self.find_user(patient_id)

        return Patient(u['last_name'], u['first_name'], None, u['address'], u['city'], u['state'], u['zipcode'], u['dob'], u['gender'])
    #
    # Very IMPORTANT
    # That HPSJ_ID that is None right now gets defined later
    #
        
    def get_responded_date_and_sr(self,chat_id): 
        important_indices = np.where(self.chat_data['session_id'].to_numpy() == chat_id)
        chat_ppl_ids = []
        for i in important_indices:
            chat_ppl_ids.append(self.chat_data['from_id'].to_numpy()[i])
        chat_role_in_order = []
        # print(chat_ppl_ids)
        for i in chat_ppl_ids[0]:
            chat_role_in_order.append(self.find_role(i))
        try:
            a = chat_role_in_order.index('Responder Provider')
        except:
            # print(chat_role_in_order)
            return None
        d = {}
        d['First'] = self.find_user(chat_ppl_ids[0][a])['first_name']
        d['Last'] = self.find_user(chat_ppl_ids[0][a])['last_name']
        d['NPI'] = self.find_user(chat_ppl_ids[0][a])['npi']
        return [self.chat_data['time'].to_numpy()[a],d]
        
    def get_chats(self):
        return self.sql_to_df('chat_sessions')
    
    def __exit__(self, *args):
       #Close connection to SQL server is server closed
        self.cursor.close()
        self.conn.close()
        print("Connection closed to SQL server")

In [2]:
class Patient:
    def __init__(self, patient_last, patient_first, hpsj_id, 
                 patient_address, patient_city, patient_state, patient_zip, 
                 patient_dob, patient_gender):
        self.last = patient_last
        self.first = patient_first
        self.id = hpsj_id
        self.address = patient_address
        self.city = patient_city
        self.state = patient_state
        self.zip = patient_zip
        import pandas as pd
        ts = pd.to_datetime(str(patient_dob))
        self.dob = ts.strftime('%Y%m%d')
        if(int(patient_gender) == 0):
            self.gender = 'F'
        elif(int(patient_gender) == 1):
            self.gender = 'M'
        else:
            self.gender = 'U'

In [3]:
class Transaction:
    
    # *** num of lines is currently 32 ***
    
    num_of_lines = 32
    
    def __init__(self, count, iday, itime, patient_obj, consult_id, responded_date,
                 icd10_code, sr_last, sr_first, sr_npi, CCI_BLANK_or_MED):
        self.data = f'ST*837*{count}*005010X222A1*~\nBHT*0019*00*0123*{iday}*{itime}*RP*~\nNM1*41*2*AFYA GLOBAL INC*****46*1982269031*~\nPER*IC*AFYA SUPPORT*TE*9493995380*EM*admin@afyachat.com*~\nNM1*40*2*HPSJ*****46*00303*~\nHL*1**20*1*~\nPRV*BI*PXC*261QM1300X*~\nNM1*85*2*HUBMD*****XX*1982269031*~\nN3*104 E Olive Ave*Suite 100*~\nN4*Redlands*CA*92373-5255*~\nREF*EI*83-2774928*~\nPER*IC*Stanley Frencher*TE*9094559473*~\nHL*2*1*22*0*~\nSBR*S*18*******MC*~\nNM1*IL*1*{patient_obj.last}*{patient_obj.first}****MI*{patient_obj.id}*~\nN3*{patient_obj.address}**~\nN4*{patient_obj.city}*{patient_obj.state}*{patient_obj.zip}*~\nDMG*D8*{patient_obj.dob}*{patient_obj.gender}*~\nNM1*PR*2*HPSJ*****PI*00303*~\nN3*7751 S Manthey Rd**~\nN4*French Camp*CA*95231*~\nCLM*{consult_id}*34.50***02:B:01*Y*A*Y*Y*~\nDTP*304*D8*{responded_date}*~\nCN1*05*0*~\nHI*ABK:{icd10_code}*~\nNM1*82*1*{sr_last}*{sr_first}****XX*{sr_npi}*~\nSBR*P*18**{CCI_BLANK_or_MED}*****MC*~\nAMT*A8*0*~\nLX*1*~\nSV1*99451:GQ*34.50*UN*1*02***~\nDTP*472*D8*20200226*~\nSE*32*{count}*~\n'

In [43]:
#
# @dev
# 
# input: Custom-made Connection object, Pandas Series for that 'row'
# output: Transaction Object
#
# need to define count, iday, itime, patient_obj, consult_id, responded_date,
#                 icd10_code, sr_last, sr_first, sr_npi, CCI_BLANK_or_MED
# 
# 
def make_transaction(c, num, chat_session):
    from datetime import date
    from datetime import datetime

    today = date.today()
    now = datetime.now()
    iday = today.strftime("%y%m%d")
    itime = now.strftime("%H%M")
    
    the_patient = c.make_patient(chat_session['patient_id'])
    
    # Consult ID - ask Alvin if there is a conversion?
    # From '53c213f0-ca29-11eb-8fc4-d1320f706d01' to something?
    
    # Ask Alvin what the responded date is?
    # Options - Index(['id', 'status', 'close_codes', 'speciality', 'permissions', 'createdAt',
    #   'updatedAt', 'members', 'chief_complaint', 'owner_id_1', 'owner_id_2',
    #   'last_sender_name', 'last_sent_message', 'last_sent_type', 'read_by',
    #   'agora_ids', 'last_sender_id', 'updatedTime', 'patient_id',
    #   'patient_agora_id', 'org_id', 'ICD', 'CPT', 'is_archive',
    #   'close_code_date', 'language'],
    #  dtype='object')
    values = c.get_responded_date_and_sr(chat_session['id'])
    if(values == None):
        return None
    import pandas as pd
    ts = pd.to_datetime(str(values[0])) 
    the_responded_date = ts.strftime('%Y%m%d')
    the_sr_last = values[1]['Last']
    the_sr_first = values[1]['First']
    the_sr_npi = values[1]['NPI']
    
    the_icd = chat_session['ICD'][:chat_session['ICD'].find(' ')]
    
    # How to extract SR?
    # Get owners? Compare them?
    # Can't tell from user list who the PCP is or SR
    # Options - Index(['user_id', 'MPI', 'first_name', 'last_name', 'dob', 'address',
    #   'createdAt', 'updatedAt', 'email', 'mobile', 'password', 'gender',
    #   'MRN', 'middle_name', 'registration_type_id', 'hospital_name',
    #   'image_url', 'spec_id', 'about', 'user_code', 'card_val', 'home_phone',
    #   'city', 'state', 'country', 'zipcode', 'org_id', 'experience', 'status',
    #   'role_id', (all of them were 0) 'insurance', 'employer', 'agora_id', 'token', 'device',
    #   'assign_time', 'fire_token', 'dobi', 'bio', 'live_status', 'npi',
    #   'medical_group', 'amount', 'volume', 'response_time',
    #   'non_face_to_face', 'CIN'],
    # dtype='object')
    
    # Also, can't tell CCI_BLANK_or_MED
#
#
#
#
#
    return Transaction(num, iday, itime, the_patient, None, the_responded_date,
                 the_icd, the_sr_last, the_sr_first, the_sr_npi, '')

In [89]:
test_connection = Connection('afyaproduction')
test_chat_session = test_connection.get_chats().loc[120:172]

In [90]:
len(test_chat_session)

53

In [91]:
c = 1
dat = []
for x in np.arange(len(test_chat_session)):
    try:
        dat.append(make_transaction(test_connection, c, test_chat_session.iloc[x]).data)
        c += 1
    except:
        pass

<ipython-input-48-0dd4f87916b8>:68: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  a = chat_role_in_order.index('Responder Provider')


In [92]:
c

33

In [58]:
envelope_start = everything[:2]
transaction_arr = everything[2:-2]
envelope_end = everything[-2:]

In [73]:
temp = ''
for e in envelope_end[1]:
    temp += str(e) + '*'

In [74]:
temp

'IEA*1*{interchange_control_number }*'

In [84]:
def create_file(transactions):
    from datetime import date
    from datetime import datetime
    from random import uniform
    today = date.today()
    now = datetime.now()
    iday = today.strftime("%y%m%d")
    itime = now.strftime("%H%M")
    num_of_transactions = len(transactions)
    interchange_control_number = str(int(uniform(1,999999999))).zfill(9)
    group_control_number = str(int(uniform(1,999999999))).zfill(9)
    info = f'ISA*00*          *00*          *ZZ*221596*ZZ*ZIRMED*{iday}*{itime}*^*00501*{interchange_control_number}*1*P*:~\n'
    info += f'GS*HC*221596*ZIRMED*{iday}*{itime}*{group_control_number}*X*005010X222A1*\n'
    for t in transactions:
        info += t
    info += f'GE*{num_of_transactions}*{group_control_number}*\n'
    info += f'IEA*1*{interchange_control_number}*'
    os.chdir('C:/Users/Melton/OneDrive/Documents/Afya/HPSJ')
    f = open(f"test_batch_HPSJ_{interchange_control_number}.edi", "w")
    f.write(info)
    f.close()

In [85]:
create_file(dat)

In [20]:
test_connection.users.columns

Index(['user_id', 'MPI', 'first_name', 'last_name', 'dob', 'address',
       'createdAt', 'updatedAt', 'email', 'mobile', 'password', 'gender',
       'MRN', 'middle_name', 'registration_type_id', 'hospital_name',
       'image_url', 'spec_id', 'about', 'user_code', 'card_val', 'home_phone',
       'city', 'state', 'country', 'zipcode', 'org_id', 'experience', 'status',
       'role_id', 'insurance', 'employer', 'agora_id', 'token', 'device',
       'assign_time', 'fire_token', 'dobi', 'bio', 'live_status', 'npi',
       'medical_group', 'amount', 'volume', 'response_time',
       'non_face_to_face', 'CIN', 'paypal_customer_id', 'stripe_customer_id',
       'member_identity'],
      dtype='object')

In [28]:
test_connection.chat_data.columns

Index(['from_id', 'from_name', 'message', 'link', 'img_url', 'updatedAt', 'id',
       'session_id', 'speciality', 'datatype', 'createdAt', 'time',
       'shortlink', 'trans_message', 'language'],
      dtype='object')

## Workflow for finding first responded 
### chat_session_id --> messages --> user_ids --> role_map --> roles --> find 2nd SR index --> map back to message time

In [22]:
t = test_connection.chat_data['session_id'].to_numpy()
c = 0
important_indices = np.where(t == 'a6deb140-d534-11eb-975f-63ef74eb6968')
test_chat = []
for i in important_indices:
    test_chat.append(test_connection.chat_data['message'].to_numpy()[i])

In [23]:
test_chat

[array(["This chat has been assigned to Dr. Lori  Hobbs (Dermatology) and the provider will respond to this chat after they've reviewed",
        'SuperAdmin has added  Dr.HarryDao to the chat',
        'Hello Dr. Dao, I hope you were able to log into the account. ',
        'able to log on - all clear!',
        'Thats great!. feel free to try the app and if you have any questions please let me know',
        'will do!'], dtype=object)]

In [108]:
test_connection.roles.columns

Index(['role_id', 'role_name', 'createdAt', 'updatedAt', 'image_url',
       'description'],
      dtype='object')

In [109]:
test_connection.role_map.columns

Index(['id', 'org_id', 'roles', 'createdAt', 'updatedAt', 'provider_id'], dtype='object')

In [110]:
# Test on last session
test_chat_session = test_connection.get_chats().loc[173]
# Pull information on last sessions

In [87]:
# Make a Patient Object


### 837P Formatting

In [ ]:
import pandas as pd

df = pd.read_excel('Updated_HPSJ837PFieldDescriptionsMeltonEd.xlsx', engine='openpyxl')

In [ ]:
df

In [54]:
def isnan(x):
    try:
        y = float(x)
        return math.isnan(y)
    except:
        return False

In [55]:
import math

test = []
arr = []
for e in df['Example']:
    if(isnan(e)):
        test.append(arr)
        arr = []
    else:
        if(e == '{BLANK}'):
            arr.append('')
        else:
            arr.append(e)


In [56]:
everything = test[1:]

In [57]:
envelope_start = everything[:2]
transaction_arr = everything[2:-2]
envelope_end = everything[-2:]

In [26]:
len(transaction_arr)

32

In [27]:
transaction = ''
newline = '\n'
for t in transaction_arr:
    for i,_t in enumerate(t):
        if i != (len(t)):
            transaction += str(_t)
            transaction += '*'
    transaction += f'~{newline}'

In [28]:
transaction

'ST*837*{count}*005010X222A1*~\nBHT*0019*00*0123*{iday}*{itime}*RP*~\nNM1*41*2*AFYA GLOBAL INC*****46*1982269031*~\nPER*IC*AFYA SUPPORT*TE*9493995380*EM*admin@afyachat.com*~\nNM1*40*2*HPSJ*****46*00303*~\nHL*1**20*1*~\nPRV*BI*PXC*261QM1300X*~\nNM1*85*2*HUBMD*****XX*1982269031*~\nN3*104 E Olive Ave*Suite 100*~\nN4*Redlands*CA*92373-5255*~\nREF*EI*83-2774928*~\nPER*IC*Stanley Frencher*TE*9094559473*~\nHL*2*1*22*0*~\nSBR*S*18*******MC*~\nNM1*IL*1*{patient_last}*{patient_first}****MI*{hpsj_id}*~\nN3*{patient_address}**~\nN4*{patient_city}*{patient_state}*{patient_zip}*~\nDMG*D8*{patient_dob}*{patient_gender}*~\nNM1*PR*2*HPSJ*****PI*00303*~\nN3*7751 S Manthey Rd**~\nN4*French Camp*CA*95231*~\nCLM*{consult_id}*34.50***02:B:01*Y*A*Y*Y*~\nDTP*304*D8*{responded_date}*~\nCN1*05*0*~\nHI*ABK:{icd10_code}*~\nNM1*82*1*{sr_last}*{sr_first}****XX*{sr_npi}*~\nSBR*P*18**{CCI_BLANK_or_MED}*****MC*~\nAMT*A8*0*~\nLX*1*~\nSV1*99451:GQ*34.50*UN*1*02***~\nDTP*472*D8*20200226*~\nSE*{num_of_lines}*{count}*~\n'

In [29]:
print(transaction)

ST*837*{count}*005010X222A1*~
BHT*0019*00*0123*{iday}*{itime}*RP*~
NM1*41*2*AFYA GLOBAL INC*****46*1982269031*~
PER*IC*AFYA SUPPORT*TE*9493995380*EM*admin@afyachat.com*~
NM1*40*2*HPSJ*****46*00303*~
HL*1**20*1*~
PRV*BI*PXC*261QM1300X*~
NM1*85*2*HUBMD*****XX*1982269031*~
N3*104 E Olive Ave*Suite 100*~
N4*Redlands*CA*92373-5255*~
REF*EI*83-2774928*~
PER*IC*Stanley Frencher*TE*9094559473*~
HL*2*1*22*0*~
SBR*S*18*******MC*~
NM1*IL*1*{patient_last}*{patient_first}****MI*{hpsj_id}*~
N3*{patient_address}**~
N4*{patient_city}*{patient_state}*{patient_zip}*~
DMG*D8*{patient_dob}*{patient_gender}*~
NM1*PR*2*HPSJ*****PI*00303*~
N3*7751 S Manthey Rd**~
N4*French Camp*CA*95231*~
CLM*{consult_id}*34.50***02:B:01*Y*A*Y*Y*~
DTP*304*D8*{responded_date}*~
CN1*05*0*~
HI*ABK:{icd10_code}*~
NM1*82*1*{sr_last}*{sr_first}****XX*{sr_npi}*~
SBR*P*18**{CCI_BLANK_or_MED}*****MC*~
AMT*A8*0*~
LX*1*~
SV1*99451:GQ*34.50*UN*1*02***~
DTP*472*D8*20200226*~
SE*{num_of_lines}*{count}*~



"
Expected Values (Dual Members): Subscriber Group Name (2000B - SBR04) & Claim Filing Indicator Code (2000B - SBR09)

1.	SBR*S*18**CMC*****MB   (CMC/MB = Medicare Professional)
a.	Expected when Dual member claims are adjudicated under the Medicare line of business.
b.	Warning: Failure to populate the expected value in the 2000B_SBR04 segment will result in encounter data rejections. 

2.	SBR*S*18**CMC*****MA   (CMC/MA = Medicare Institutional)
a.	Expected when dual member claims are adjudicated under the Medicare line of business.
b.	Warning: Failure to populate the expected value in the 2000B_SBR04 segment will result in encounter data rejections. 


3.	SBR*S*18**CCI*****MC   (CCI/MC = Medicaid Professional and/or Institutional)
a.	Expected when Dual member claims are adjudicated under the Medicaid line of business.
b.	Warning: Failure to populate the expected value in the 2000B_SBR04 segment will result in encounter data rejections. 


Expected Values (Medicaid Members): Subscriber Group Name (2000B - SBR04) & Claim Filing Indicator Code (2000B - SBR09)

1.	SBR*S*18*******MC   ([Blank – SBR04]/MC = Medicaid Professional and/or Institutional)
a.	Expected when Medicaid member claims are adjudicated under the Medicaid line of business.
b.	Warning: Failure to populate the expected value in the 2000B_SBR04 segment will result in encounter data rejections. 

2.	SBR*S*18**MED *****MC   (MED/MC = Medicaid Professional and/or Institutional)
a.	Expected when dual member claims are adjudicated under the Medicare contracted rates.
b.	Warning: Failure to populate the expected value in the 2000B_SBR04 segment will result in encounter data rejections. 

3.	Note: Both options 1 & 2 are acceptable for claims adjudicated under the non-dual Medicaid line of business. "